# Data Owner


## Upload Dataset

In [ ]:
from pathlib import Path

from huggingface_hub import snapshot_download

DATASET_DIR = Path("./dataset/").expanduser().absolute()

if not DATASET_DIR.exists():
    snapshot_download(
        repo_id="khoaguin/pima-indians-diabetes-database-partitions",
        repo_type="dataset",
        local_dir="./dataset/",
    )

In [ ]:
LOCAL_TEST = True
DO_EMAIL = "flower-test-group-2@openmined.org"

if LOCAL_TEST:
    from syft_rds.orchestra import setup_rds_server

    do_stack = setup_rds_server(email=DO_EMAIL, root_dir=Path("."), key="flwr")
    do_client = do_stack.init_session(host=DO_EMAIL)
else:
    import syft_rds as sy

    do_client = sy.init_session(host=DO_EMAIL)

In [ ]:
assert do_client.is_admin

In [ ]:
do_client.datasets

In [ ]:
FLWR_PROJECT = "fed-analytics-diabetes"
SYFTBOX_DATASET_NAME = "pima-indians-diabetes-database"
PARTITION_ID = 2

DATASET_DIR = Path("./dataset/").expanduser().absolute()
FLWR_PROJECT_PATH = Path(f"{FLWR_PROJECT}")

DATASET_PRIVATE_PATH = (
    DATASET_DIR / f"{SYFTBOX_DATASET_NAME}-{PARTITION_ID}" / "private"
)
DATASET_MOCK_PATH = DATASET_DIR / f"{SYFTBOX_DATASET_NAME}-{PARTITION_ID}" / "mock"
README_PATH = DATASET_DIR / f"{SYFTBOX_DATASET_NAME}-{PARTITION_ID}" / "README.md"

In [ ]:
try:
    dataset = do_client.dataset.create(
        name=SYFTBOX_DATASET_NAME,
        summary=f"Partitioned {SYFTBOX_DATASET_NAME} dataset.",
        description_path=README_PATH,
        path=DATASET_PRIVATE_PATH,
        mock_path=DATASET_MOCK_PATH,
    )
    dataset.describe()
except Exception as e:
    print(f"Error: {e}")

In [ ]:
do_client.datasets

## Check and Run Jobs

In [ ]:
jobs = do_client.jobs.get_all()
jobs

In [ ]:
import os

if LOCAL_TEST:
    os.environ["SYFTBOX_CLIENT_CONFIG_PATH"] = str(do_stack.client.config_path)

do_client.run_private(jobs[0])